In [1]:
# theano imports
import theano
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
from theano.tensor.nnet.conv import conv2d
# from theano.tensor.signal.downsample import max_pool_2d
from theano.tensor.signal.pool import pool_2d as max_pool_2d

# other imports
from sklearn.cross_validation import train_test_split
import numpy as np
import pandas as pd
import pickle

In [2]:
# import warnings
# warnings.filterwarnings("ignore")

In [3]:
# read our data in 
with open('conv_data/5_tensor.p', 'rb') as f:
    loaded_data = pickle.load(f)

labels = pd.read_csv('one_hot_labels.csv')

In [4]:
loaded_data[:2]    

array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])

In [5]:
one_hot = labels.values

In [6]:
one_hot[:2]

array([[0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [7]:
xTrain, xTest, yTrain, yTest = train_test_split(loaded_data, one_hot)

In [8]:
print xTrain.shape, xTest.shape, yTrain.shape, yTest.shape

(101110, 5, 21) (33704, 5, 21) (101110, 6) (33704, 6)


In [9]:
xTrain = xTrain.reshape(-1, 1, 5, 21)
xTest = xTest.reshape(-1, 1, 5, 21)

In [10]:
print xTrain.shape
print xTest.shape
print yTrain.shape
print yTest.shape

(101110, 1, 5, 21)
(33704, 1, 5, 21)
(101110, 6)
(33704, 6)


In [48]:
srng = RandomStreams()

def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def activate(X):
    return T.nnet.relu(X)

def rectify(X):
    return T.maximum(X, 0.)
    #return T.maximum(X, 0.01*X)  #leaky rectifier

def softmax(X):
    e_x = T.exp(X - X.max(axis=1).dimshuffle(0, 'x'))
    return e_x / e_x.sum(axis=1).dimshuffle(0, 'x')

def dropout(X, p=0.0):
    if p > 0:
        retain_prob = 1 - p
        X *= srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
        X /= retain_prob
    return X

def RMSprop(cost, params, lr=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    
    return updates

def model(X, wi, wh, wo, p_drop_conv, p_drop_hidden):
    layer_1 = rectify(conv2d(X, wi)) # , border_mode='full'
#     layer_1 = dropout(layer_1, p_drop_conv)

    layer_2 = rectify(T.dot(layer_1, wh))
#     layer_2 = dropout(layer_2, p_drop_hidden)

#     pyx = softmax(T.dot(layer_2, wo))
    pyx = T.nnet.softmax(T.dot(layer_2, wo))
    return layer_1, layer_2, pyx

In [49]:
# def model(X, w, w2, w3, w4, p_drop_conv, p_drop_hidden):
#     l1a = rectify(conv2d(X, w)) # , border_mode='full'
# #     l1 = max_pool_2d(l1a, (2, 2))
#     l1 = dropout(l1a, p_drop_conv)

#     l2a = rectify(conv2d(l1, w2))
# #     l2 = max_pool_2d(l2a, (2, 2))
#     l2 = dropout(l2a, p_drop_conv)

#     l3a = rectify(conv2d(l2, w3))
# #     l3b = max_pool_2d(l3a, (2, 2))
#     l3 = T.flatten(l3a, outdim=2)
#     l3 = dropout(l3, p_drop_conv)

#     l4 = rectify(T.dot(l3, w4))
#     l4 = dropout(l4, p_drop_hidden)

#     pyx = softmax(T.dot(l4, w_o))
#     return l1, l2, l3, l4, pyx

# w = init_weights((32, 1, 3, 3))
# w2 = init_weights((64, 32, 3, 3))
# w3 = init_weights((128, 64, 3, 3))
# w4 = init_weights((128 * 3 * 3, 625))
# w_o = init_weights((625, 10))

In [50]:
X = T.ftensor4()
Y = T.fmatrix()

wi = init_weights((128, 1, 5, 21))
wh = init_weights((128, 600))
wo = init_weights((600, 6))

# noise_l1, noise_l2, noise_py_x = model(X, wi, wh, wo, 0.2, 0.5)
l1, l2, py_x = model(X, wi, wh, wo, 0., 0.)
y_x = T.argmax(py_x, axis=1)


cost = T.mean(T.nnet.categorical_crossentropy(noise_py_x, Y))
params = [wi, wh, wo]
updates = RMSprop(cost, params, lr=1e-7)

train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)
predict = theano.function(inputs=[X], outputs=y_x, allow_input_downcast=True)

# number of training iterations to perform
n_train = 11

# define mini-batch size
mbs = 128

ValueError: x must be 1-d or 2-d tensor of floats. Got TensorType(float32, 4D)

In [27]:
# performing our training
for i in xrange(n_train):
    for start, end in zip(xrange(0, len(xTrain), mbs), 
                          xrange(mbs, len(xTrain), mbs)):
        cost = train(xTrain[start:end], yTrain[start:end])
#         a, b, c, d, e = model(floatX(xTrain[start:end]), w, w2, w3, w4, 0., 0.)
#         print a.eval().shape, b.eval().shape, c.eval().shape, d.eval().shape, e.eval().shape
        print i, np.mean(np.argmax(yTest, axis=1) == predict(xTest))
    print np.mean(np.argmax(yTest, axis=1) == predict(xTest))

NameError: name 'n_train' is not defined

http://deeplearning.net/software/theano/library/tensor/nnet/nnet.html#tensor.nnet.softmax
http://deeplearning.net/software/theano/library/tensor/nnet/conv.html#theano.tensor.nnet.conv.conv2d